In [3]:
import pandas as pd
import os
import glob

# --- 1. CONFIGURACIÓN ---
BASE_DIR = '/Users/omartellez/Manzanillo'

# Apuntamos a la carpeta donde están todos los archivos consolidados
input_folder = os.path.join(BASE_DIR, '03_base_consolidada') 
# Crearemos una nueva carpeta para el resultado final
output_folder = os.path.join(BASE_DIR, '04_base_maestra')
output_file_csv = 'elecciones_manzanillo_master_2012_2024.csv'
output_file_parquet = 'elecciones_manzanillo_master_2012_2024.parquet' # Formato más eficiente (opcional)

# --- 2. ENCONTRAR Y CARGAR TODOS LOS ARCHIVOS CONSOLIDADOS ---
search_pattern = os.path.join(input_folder, '*.csv')
consolidated_files = sorted(glob.glob(search_pattern))

if not consolidated_files:
    print("No se encontraron archivos consolidados para unir.")
else:
    all_dfs = []
    print(f"Se encontraron {len(consolidated_files)} archivos consolidados para unir:")
    for file_path in consolidated_files:
        print(f"  -> Cargando: {os.path.basename(file_path)}")
        df = pd.read_csv(file_path)
        all_dfs.append(df)

    # --- 3. CONSOLIDACIÓN FINAL, ORDENAMIENTO Y GUARDADO (VERSIÓN CORREGIDA) ---
# La unión maestra
master_df = pd.concat(all_dfs, ignore_index=True)

# Lógica de ordenamiento automático y robusto (se mantiene igual)
columnas_inicio = [
    'seccion', 'casilla', 'anio', 'tipo_eleccion', 'lista_nominal',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados'
]
columnas_fin = ['voto_no_registrado', 'voto_nulo', 'voto_total']
columnas_inicio_existentes = [col for col in columnas_inicio if col in master_df.columns]
columnas_fin_existentes = [col for col in columnas_fin if col in master_df.columns]
columnas_votos_crudos = sorted([
    col for col in master_df.columns 
    if col not in columnas_inicio_existentes and col not in columnas_fin_existentes
])
orden_final = columnas_inicio_existentes + columnas_votos_crudos + columnas_fin_existentes
master_df_ordenado = master_df[orden_final]

# --- NUEVA LÓGICA DE RELLENO INTELIGENTE ---
# 1. Rellenar columnas de texto (object) con un marcador de texto
object_cols = master_df_ordenado.select_dtypes(include='object').columns
master_df_ordenado[object_cols] = master_df_ordenado[object_cols].fillna('SIN_DATO')

# 2. Rellenar el resto de las columnas (que serán numéricas) con 0
master_df_final = master_df_ordenado.fillna(0)

# 3. Convertir columnas de votos que puedan ser float a int para limpieza final
for col in master_df_final.columns:
    if 'voto' in col or 'coalition' in col or '_votos' in col or 'margen' in col:
        # Chequeamos si la columna es numérica antes de convertir
        if pd.api.types.is_numeric_dtype(master_df_final[col]):
            master_df_final[col] = master_df_final[col].astype(int)

# --- FIN DE LA NUEVA LÓGICA ---
    
# Guardar la base de datos maestra
os.makedirs(output_folder, exist_ok=True)

# Guardar en CSV
output_path_csv = os.path.join(output_folder, output_file_csv)
master_df_final.to_csv(output_path_csv, index=False)
print(f"\n¡Base de datos Maestra (CSV) creada exitosamente!")
print(f"Guardada en: {output_path_csv}")

# Guardar en Parquet (ahora funcionará)
try:
    output_path_parquet = os.path.join(output_folder, output_file_parquet)
    master_df_final.to_parquet(output_path_parquet, index=False)
    print(f"\n¡Base de datos Maestra (Parquet) creada exitosamente!")
    print(f"Guardada en: {output_path_parquet}")
except ImportError:
    print("\nSugerencia: para guardar en formato Parquet, instala 'pyarrow'.")

print("\nResumen de la Base de Datos Maestra:")
master_df_final.info()

Se encontraron 6 archivos consolidados para unir:
  -> Cargando: ayuntamiento_consolidado_2012_2024.csv
  -> Cargando: diputado_federal_consolidado_2012_2024.csv
  -> Cargando: diputados_locales_consolidado_2012_2024.csv
  -> Cargando: gobernador_consolidado_2016_2021.csv
  -> Cargando: presidente_consolidado_2012_2024.csv
  -> Cargando: senadores_consolidado_2012_2024.csv

¡Base de datos Maestra (CSV) creada exitosamente!
Guardada en: /Users/omartellez/Manzanillo/04_base_maestra/elecciones_manzanillo_master_2012_2024.csv

¡Base de datos Maestra (Parquet) creada exitosamente!
Guardada en: /Users/omartellez/Manzanillo/04_base_maestra/elecciones_manzanillo_master_2012_2024.parquet

Resumen de la Base de Datos Maestra:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 70 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   seccion                  5515 non-null   int64  
 1   

In [5]:
print("\nRealizando conversión final de tipos de dato (float a int)...")

for col in master_df_final.columns:
    # Si la columna es de tipo float...
    if master_df_final[col].dtype == 'float64':
        # ...y no tiene valores decimales reales, la convertimos a entero.
        # Esto es seguro porque sabemos que son conteos de votos.
        master_df_final[col] = master_df_final[col].astype('int64')

print("\n¡Conversión de tipos de dato finalizada!")
print("\nResumen definitivo de la Base de Datos Maestra:")
master_df_final.info()


Realizando conversión final de tipos de dato (float a int)...

¡Conversión de tipos de dato finalizada!

Resumen definitivo de la Base de Datos Maestra:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 70 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  5515 non-null   int64 
 1   casilla                  5515 non-null   object
 2   anio                     5515 non-null   int64 
 3   tipo_eleccion            5515 non-null   object
 4   lista_nominal            5515 non-null   int64 
 5   ganador_casilla_partido  5515 non-null   object
 6   ganador_casilla_votos    5515 non-null   int64 
 7   segundo_lugar_partido    5515 non-null   object
 8   segundo_lugar_votos      5515 non-null   int64 
 9   margen_victoria          5515 non-null   int64 
 10  morena_coalitions        5515 non-null   int64 
 11  non_morena_coalitions    5515 non-null   int6